# DNA and Bone
# 2/3/2564 Edit

## 10 input && 2 output
## 28x24 dataset

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#conda install -c anaconda xlrd

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

# pip install git+https://github.com/tensorflow/docs

In [ ]:
## Input
columns_input = ['Allele_1', 'Allele_2', 'Allele_3', 'Allele_4', 'Allele_5',
                  'Height_1', 'Height_2', 'Height_3', 'Height_4', 'Height_5']
## Output
columns_output = ['Out_1', 'Out_2']

print(len(columns_input))
print(len(columns_output))

In [ ]:
columns_names = []
columns_names = columns_input + columns_output

In [ ]:
print(columns_names[0])
print(columns_names[5])
print(columns_names[8])
print(columns_names[9])

In [ ]:
file_name = 'new_V2_mix_major_Full.xlsx'
path_adress = "C:\\Users\\MSI-1016TH\\Google Drive\\DNA_project_Code\\"
dataset_path = path_adress + file_name

In [ ]:
sheet1 = 'Sheet4'
cols = 'c:l'
df_input = pd.read_excel(dataset_path, sheet_name=sheet1, header=None, skiprows=[0,1], usecols=cols)
cols = 'm:n'
df_output = pd.read_excel(dataset_path, sheet_name=sheet1, header=None, skiprows=[0,1], usecols=cols)
cols = 'c:n'
df_all = pd.read_excel(dataset_path, sheet_name=sheet1, header=None, skiprows=[0,1], usecols=cols)

#df_input
df_all.head()
#df_output

#df.head()
#df.tail()

In [ ]:
df_all = df_all.fillna(0) # แทนค่า nan = 0
df_all = df_all.replace('X', 1)
df_all = df_all.replace('Y', 2)

In [ ]:
df_all.head()

In [ ]:
## All

major = 28 # sample
marker_rows = 24
marker_columns = 10+2

lst_all = []
num_columns = marker_columns # newcolumns
lst_all = [0] * (major*marker_rows) # Generate Rows
for i in range(len(lst_all)):
    lst_all[i] = [0] * num_columns

for k in range(0,major):
    count = k * marker_rows
    lst = []
    df_temp = df_all[24*k:(24*k)+24]
    array_temp = df_temp.to_numpy()
    lst = array_temp.tolist()
    del df_temp, array_temp
    for i in range(0,marker_rows):
        for j in range(0,marker_columns):
            #lst_all[count][j] = lst[i][j]
            if j < 4 :
                lst_all[count][j] = lst[i][j]/100
            elif j >= 5 and j < 10:
                lst_all[count][j] = lst[i][j]/10000
            elif j >= 10 and j < 12:
                lst_all[count][j] = lst[i][j]/100
        count = count+1

In [ ]:
print(len(lst_all))     #rows
print(len(lst_all[0]))  #columns
print(len(lst_all[10]))  #columns

In [ ]:
count

In [ ]:
lst_all

In [ ]:
dataset_all = pd.DataFrame(lst_all, columns=columns_names)
dataset_all.head()

In [ ]:
dataset_all

In [ ]:
dataset_all[0:605]

In [ ]:
dataset = dataset_all.copy()
train_dataset = dataset.sample(frac=0.9, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
print(train_dataset.shape)
print(test_dataset.shape)

In [ ]:
print(columns_input)
print(columns_output)

In [ ]:
#df.drop('column_name', 1)
train_data = train_dataset.drop(columns=columns_output)
train_labels = train_dataset.drop(columns=columns_input)

test_data = test_dataset.drop(columns=columns_output)
test_labels = test_dataset.drop(columns=columns_input)

In [ ]:
train_data
#df.head()
#df.tail()

In [ ]:
train_data[1:5]

In [ ]:
train_data.tail()

In [ ]:
train_labels.head()

In [ ]:
test_data[0:15]

In [ ]:
#test_labels.head()
(test_labels*100)[0:15]

In [ ]:
# สร้าง neural network โดยใข้  Library keras

def build_model():
    # define the model ระบุโมเดล
    model = keras.Sequential([
        layers.Dense(2048, activation='relu', input_shape=[len(train_data.keys())]),
        layers.Dense(1024, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(2)
    ])
    
    # define optimizer ระบุ optimizer
    #optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam(0.001)
    
    # สร้าง model และระบุ loss function แบบ mean square error
    model.compile(loss='mae',
                  optimizer=optimizer, 
                  metrics=['mae','mse'])
    return model

In [ ]:
len(train_data.keys())

In [ ]:
model = build_model() # สร้าง model

In [ ]:
model.summary()

In [ ]:
example_result = model.predict(train_data[:5]) # ทดสอบดูว่า model เราสามารถทำงานได้
example_result

In [ ]:
# ทำการ training โมเดล โดยแบ่งข้อมูลสำหรับ test เป็น 10% จากชุดข้อมูล
EPOCHS = 500

history = model.fit(
    train_data, train_labels,
    epochs=EPOCHS, validation_split=0.1, verbose=0,
    callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2) #สร้าง 0bject plotter

In [ ]:
# plot ดูผล accuracy ของ training
plotter.plot({'Basic':history}, metric = 'mae')
plt.ylim([0,10])
plt.ylabel('MAE [mpg]')

In [ ]:
# plot ดูผล Mean square error ของ training
plotter.plot({'Basic':history}, metric = 'mse')
plt.ylim([0,10])
plt.ylabel('MSE [mpg]')

In [ ]:
loss, mae, mse = model.evaluate(test_data, test_labels, verbose=2) 
# ประเมิน โดยการทดสอบกับข้อมูลที่มันไม่เคยเห็น

print("Testing set Mean Abs Error: {:5.2f} mpg".format(mae))

In [ ]:
# run ข้อมูล prediction จาก model ที่ train แล้ว 
test_predictions = model.predict(test_data)
#test_predictions = test_predictions.flatten() # flatten ให้เป็น array 1 มิติ
test_predictions*100
#type(test_predictions)

In [ ]:
# Prepare Data Before Compare with Input
predictions_data = test_predictions*100
array_test_data = (test_data*100).to_numpy()

#print(type(array_test_data))
#print(type(predictions_data))


In [ ]:
# compare with Input
Perd_labels = np.zeros((67, 2))

for k in range(0,len(array_test_data)):
    for i in range(0,2):
        c = np.array([99])
        c_new = np.array([0])
        for j in range(0,5):
            c_new = abs(predictions_data[k, i] - array_test_data[k, j])
            #print(c_new)
            if c_new < c:
                Perd_labels[k, i] = array_test_data[k, j]
                c = c_new

In [ ]:
Perd_labels

In [ ]:
predictions_data = pd.DataFrame({'Pred_Out_1': Perd_labels[:, 0], 'Pred_Out_2': Perd_labels[:, 1]})
predictions_data[0:15]

In [ ]:
#test_labels*100
array_test_labels = (test_labels*100).to_numpy()
array_test_labels

In [ ]:
# เปรียบเทียบกับค่าที่ไม่เคยสอน
count_0_out = 0
count_1_out = 0
count_2_out = 0
for k in range(0,67):
    for i in range(0,2):
        chk_1 = array_test_labels[k, 0] - Perd_labels[k, 0]
        chk_2 = array_test_labels[k, 1] - Perd_labels[k, 1]
    if chk_1 == 0 and chk_2 == 0:
         count_2_out = count_2_out + 1
    if chk_1 == 0 or chk_2 == 0:
         count_1_out = count_1_out + 1
    if not(chk_1 == 0 or chk_2 == 0):
         count_0_out = count_0_out + 1

print("Predictions 2 Out :", count_2_out)
print("Predictions 2 Out :", (count_2_out/67)*100, "%")
print("Predictions 1 Out :",count_1_out-count_2_out)
print("Predictions 1 Out :", ((count_1_out-count_2_out)/67)*100, "%")
print("Predictions 0 Out :",count_0_out)
print("Predictions 0 Out :", (count_0_out/67)*100, "%")

In [ ]:
# เปรียบเทียบกับค่าที่เคยสอน
#train_data
#train_labels
array_train_labels = (train_labels*100).to_numpy()
array_train_labels

# run ข้อมูล prediction จาก model ที่ train แล้ว 
train_predictions = model.predict(train_data)
#test_predictions = test_predictions.flatten() # flatten ให้เป็น array 1 มิติ
#test_predictions*100
#type(test_predictions)

# Prepare Data Before Compare with Input
predictions_data = train_predictions*100
array_train_data = (train_data*100).to_numpy()

#print(type(array_test_data))
#print(type(predictions_data))


In [ ]:
# เปรียบเทียบกับค่าที่เคยสอน
# compare with Input
Perd_labels = np.zeros((605, 2))

for k in range(0,len(array_train_data)):
    for i in range(0,2):
        c = np.array([99])
        c_new = np.array([0])
        for j in range(0,5):
            c_new = abs(predictions_data[k, i] - array_train_data[k, j])
            #print(c_new)
            if c_new < c:
                Perd_labels[k, i] = array_train_data[k, j]
                c = c_new

In [ ]:
# เปรียบเทียบกับค่าที่เคยสอน

count_1_out = 0
count_2_out = 0
for k in range(0,605):
    for i in range(0,2):
        chk_1 = array_train_labels[k, 0] - Perd_labels[k, 0]
        chk_2 = array_train_labels[k, 1] - Perd_labels[k, 1]
    if chk_1 == 0 and chk_2 == 0:
         count_2_out = count_2_out + 1
    if chk_1 == 0 or chk_2 == 0:
         count_1_out = count_1_out + 1

print("Predictions 2 Out :", count_2_out)
print("Predictions 2 Out :", (count_2_out/605)*100, "%")
print("Predictions 1 Out :",count_1_out)
print("Predictions 1 Out :", (count_1_out/605)*100, "%")